Data Scraping Notebook

In [ ]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

# Import the API key
from config import geoapify_key
from config import census_key


# Launch the browser
browser = Browser('chrome')

Create a "dictionary" list listing all states - Createing the list will help to make sure all states are listed and no state data will be lost in dataframe merges that will follow. 

In [ ]:
# Set the URL to visit to a variable
url_states = "https://developers.google.com/public-data/docs/canonical/states_csv"

# Send the browser to the URL
browser.visit(url_states)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#find all divs that contain state marker
table_state = soup.find('div', class_='devsite-table-wrapper')

In [ ]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state = {}

# Find the table element
state_list = []


# Check if the table is found
if table_state:
    # Find all rows in the table body
    rows = table_state.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            lat = cells[1].text.strip()
            lon = cells[2].text.strip()
            name = cells[3].text.strip()
            
            # Create a dictionary for the current state
            state = {'state_abbr': state, 'state': name}
            
            # Append the state data dictionary to the list
            state_list.append(state)

# Print the extracted data
#for state in state_list:
    #print(state)

In [ ]:
#load scraped state data into dataframe
state_df = pd.DataFrame(state_list)
#save dataframe to csv
state_df.to_csv('Resources/state_list.csv', index=False)

1. Which states require payment of state taxes on gambling winnings?

 A table showing the latest information regarding which states tax gambling winnings. Keep in mind that each state that requires state tax on gambling income has its own tax rate and guidelines. In some cases, gambling winnings are subject to state tax except for lottery winnings.

In [ ]:
# Set the URL to visit to a variable
url = "https://www.playusa.com/taxes/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#find all divs that contain state marker
table = soup.find('table', class_='tablepress-responsive')

In [ ]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state_tax = {}

# Find the table element
state_tax_list = []

# Find the table element
#table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if table:
    # Find all rows in the table body
    rows = table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            tax_status = cells[1].text.strip()
            
            # Create a dictionary for the current state
            state_data = {'state': state, 'tax_status': tax_status}
            
            # Append the state data dictionary to the list
            state_tax_list.append(state_data)

# Print the extracted data
for state_data in state_tax_list:
    print(state_data)

In [ ]:
#create a dataframe from the state_data dictionary of lists and display the first 5 rows
state_tax_df = pd.DataFrame(state_tax_list)
state_tax_df.head()

In [ ]:
#replace D.C. to District of Columbia in the state column
state_tax_df['state'] = state_tax_df['state'].replace('D.C.', 'District of Columbia')

Combining the tax information with the full list of states to ensure all states are listed for merges that follow. 

In [ ]:
#merge state_df and state_tax_df
state_tax_df = state_df.merge(state_tax_df, on='state', how='left')

In [ ]:
state_tax_df.to_csv('Resources/state_tax_df.csv', index=False)

2. Scrape the running total for sports betting handle and revenue reported to date across all active markets:

In [ ]:
# Set the URL to visit to a variable
url = "https://www.legalsportsreport.com/sports-betting/revenue/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
rev_table = soup.find('table', class_='tablepress-id-Rev_Markets')

In [ ]:
# Create an empty list to store market data
market_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if rev_table:
    # Find all rows in the table body
    rows = rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            market = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            market_data = {'state': market, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            market_data_list.append(market_data)

# Print the extracted data
for market_data in market_data_list:
    print(market_data)

In [ ]:
market_rev_df = pd.DataFrame(market_data_list)
market_rev_df.head(51)

In [ ]:
#replace "?"" in market_rev_df for handle, revenue, hold, and taxes with 0
market_rev_df = market_rev_df.replace({'?': 0})

Combining the tax information with the full list of states to ensure all states are listed for merges that follow. 

In [ ]:
#merge state_df and market_rev_df by state
state_rev_df = state_df.merge(market_rev_df, on='state', how='left')
#state_rev_df.head()
state_rev_df.head(51)

In [ ]:
#replace NaN with 0
state_rev_df = state_rev_df.fillna(0)

Adding the sports league detail to the revenue by state df.

In [ ]:
#add sport league info to the market revenue data frame - data added from a csv file copied from WIKI https://en.wikipedia.org/wiki/Major_professional_sports_teams_of_the_United_States_and_Canada#:~:text=Teams%20%20%20%20Team%20%20%20,%20%20MLB%20%2018%20more%20rows%20
#file previously copied directly and saved in the csv format
#read the csv file
us_major_teams_df = pd.read_csv('Resources/us_major_teams.csv')
us_major_teams_df.head()


In [ ]:
# Create a summary DataFrame to count unique teams and leagues per state
summary_data = []

# Get unique states
unique_states = us_major_teams_df['State/Province'].unique()

for state in unique_states:
    state_teams = us_major_teams_df[us_major_teams_df['State/Province'] == state]
    total_teams = state_teams['Team'].nunique()
    
    state_summary = {'state': state, 'Teams Count': total_teams}
    
    # Count teams for each league
    leagues = ['MLB', 'NBA', 'NFL', 'NHL', 'MLS', 'NCAA']
    for league in leagues:
        league_count = state_teams[state_teams['League'] == league]['Team'].nunique()
        state_summary[league] = league_count
    
    summary_data.append(state_summary)

# Create the summary DataFrame
summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
print(summary_df)



In [ ]:
#merge market_rev_df with summary_df with by state with join on state 
merged_state_rev_league_df = pd.merge(state_rev_df, summary_df, on='state', how='left')

# Display the merged DataFrame
merged_state_rev_league_df.head(51)


In [ ]:
#show datatype for each column 
merged_state_rev_league_df.dtypes

In [ ]:
# Remove '$' and ',' and convert to float
merged_state_rev_league_df['handle'] = merged_state_rev_league_df['handle'].str.replace('[\$,]', '', regex=True).astype(float)
merged_state_rev_league_df['revenue'] = merged_state_rev_league_df['revenue'].str.replace('[\$,]', '', regex=True).astype(float)
merged_state_rev_league_df['taxes'] = merged_state_rev_league_df['taxes'].str.replace('[\$,]', '', regex=True).astype(float)

# Set the float format
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
def convert_percentage(percentage_value):
    if isinstance(percentage_value, str) and percentage_value != '':
        return float(percentage_value.replace('%', '')) / 100
    else:
        return 0.0  # Handle empty strings and non-string values by returning 0.0

merged_state_rev_league_df['hold'] = merged_state_rev_league_df['hold'].apply(convert_percentage)

In [ ]:
#Datframe cleanup before loading into csv:
merged_state_rev_league_df = merged_state_rev_league_df.fillna(0)

In [ ]:
# Display the merged DataFrame
merged_state_rev_league_df.head(51)

In [ ]:
market_rev_df.to_csv('Resources/market_rev_df.csv', index=False)

3. US sports betting revenue by month

In [ ]:
month_rev_table = soup.find('table', class_='tablepress-id-Rev_Months')

In [ ]:
# Create an empty list to store market data
month_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if month_rev_table:
    # Find all rows in the table body
    rows = month_rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            month = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            timeline_data = {'month': month, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            month_data_list.append(timeline_data)

# Print the extracted data
for timeline_data in month_data_list:
    print(timeline_data)

In [ ]:
month_rev_data_df = pd.DataFrame(month_data_list)
month_rev_data_df.head(5)

In [ ]:
#data cleaning before saving the csv file
# Remove '$' and ',' and convert to float
month_rev_data_df['handle'] = month_rev_data_df['handle'].str.replace('[\$,]', '', regex=True).astype(float)
month_rev_data_df['revenue'] = month_rev_data_df['revenue'].str.replace('[\$,]', '', regex=True).astype(float)
month_rev_data_df['taxes'] = month_rev_data_df['taxes'].str.replace('[\$,]', '', regex=True).astype(float)

# Set the float format
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
def convert_percentage(percentage_value):
    if isinstance(percentage_value, str) and percentage_value != '':
        return float(percentage_value.replace('%', '')) / 100
    else:
        return 0.0  # Handle empty strings and non-string values by returning 0.0

month_rev_data_df['hold'] = month_rev_data_df['hold'].apply(convert_percentage)

In [ ]:
month_rev_data_df.head(5)

In [ ]:
month_rev_data_df.to_csv('Resources/month_rev_data_df.csv', index=False)

3. Major Professional sports teams of the US and canada - Data did not scrape correctly due to table set up in Wikipedia - single cell overlap between the states etc--> pulled data by copying from excel --> if anyone wants to try you are more than welcome to dig in --> current file major_teams_usa.csv

5. build a large data scrape loop to get each states data in a dataframe --> see if possible  

4. Add geopify API to state list --> Map Leaflet Set up --> https://www.geoapify.com/boundaries-api

5. Census population data was downloaded directly from the census tabular menu - no need to do an API. - https://data.census.gov/table - https://data.census.gov/table?q=B05003&g=010XX00US$0400000&tid=ACSDT1Y2021.B05003 - 
B05003SEX BY AGE BY NATIVITY AND CITIZENSHIP STATUS

6. Summary by online vs in-person betting by state done via download - https://www.gamingtoday.com/states/ 